# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacksongio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jacksongio/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/jacksongio/Makerspace/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/jacksongio/Makerspace/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/jacksongio/Makerspace/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '728a30'. Skipping!
Property 'summary' already exists in node 'dc88c3'. Skipping!
Property 'summary' already exists in node 'fb86a8'. Skipping!
Property 'summary' already exists in node 'b30ea5'. Skipping!
Property 'summary' already exists in node 'a72126'. Skipping!
Property 'summary' already exists in node '916f22'. Skipping!
Property 'summary' already exists in node '1ddeae'. Skipping!
Property 'summary' already exists in node '54ea40'. Skipping!
Property 'summary' already exists in node 'a4ded6'. Skipping!
Property 'summary' already exists in node '7cda74'. Skipping!
Property 'summary' already exists in node '6dfeb9'. Skipping!
Property 'summary' already exists in node '89d041'. Skipping!
Property 'summary' already exists in node '471af3'. Skipping!
Property 'summary' already exists in node 'fae463'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '728a30'. Skipping!
Property 'summary_embedding' already exists in node 'fb86a8'. Skipping!
Property 'summary_embedding' already exists in node '1ddeae'. Skipping!
Property 'summary_embedding' already exists in node 'b30ea5'. Skipping!
Property 'summary_embedding' already exists in node 'a72126'. Skipping!
Property 'summary_embedding' already exists in node '916f22'. Skipping!
Property 'summary_embedding' already exists in node 'dc88c3'. Skipping!
Property 'summary_embedding' already exists in node '54ea40'. Skipping!
Property 'summary_embedding' already exists in node '6dfeb9'. Skipping!
Property 'summary_embedding' already exists in node '471af3'. Skipping!
Property 'summary_embedding' already exists in node 'a4ded6'. Skipping!
Property 'summary_embedding' already exists in node '7cda74'. Skipping!
Property 'summary_embedding' already exists in node '89d041'. Skipping!
Property 'summary_embedding' already exists in node 'fae463'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
1. SingleHopSpecificQuerySynthesizer: Creates questions that can be answered from a single document with specific factual information.
2. MultiHopAbstractQuerySynthesizer: Creates questions requiring infromation from multiple sources to understand general concepts or relationships.
3. MultiHopSpecificQuerySynthesizer: Creates questions requiring information from multiple sources to provide specific, detailed answers.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the department?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not explicitly define what th...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about?,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is the significance of Volume 8 in relati...,[Inclusion of Clinical Work in a Standard Term...,"Volume 8, Chapter 3 provides additional guidan...",single_hop_specifc_query_synthesizer
3,What is Title IV and how does it relate to pay...,[Non-Term Characteristics A program that measu...,"Title IV programs, except for the Federal Work...",single_hop_specifc_query_synthesizer
4,Could you explain what Appendix A entails rega...,[both the credit or clock hours and the weeks ...,Appendix A provides guidance on the effect of ...,single_hop_specifc_query_synthesizer
5,disbursement timing in subscription programs a...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in clock-hour or non...,multi_hop_abstract_query_synthesizer
6,"In the academic years, what are the regulatory...","[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatory citations 34 CFR 668.3(a) and 3...,multi_hop_abstract_query_synthesizer
7,How does participation in clinical or practicu...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Participation in clinical or practicum experie...,multi_hop_abstract_query_synthesizer
8,How do Appendices A and B relate to the disbur...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A provides guidance on calculating Pe...,multi_hop_specific_query_synthesizer
9,How do the guidelines in Volume 2 and Volume 8...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines in Volume 2 specify that clinic...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7485bf'. Skipping!
Property 'summary' already exists in node 'bc1cfb'. Skipping!
Property 'summary' already exists in node 'b4b10f'. Skipping!
Property 'summary' already exists in node '05acc4'. Skipping!
Property 'summary' already exists in node 'f74c8e'. Skipping!
Property 'summary' already exists in node 'ae453a'. Skipping!
Property 'summary' already exists in node 'ea90f3'. Skipping!
Property 'summary' already exists in node 'ea06fc'. Skipping!
Property 'summary' already exists in node 'ad6808'. Skipping!
Property 'summary' already exists in node 'e10c1d'. Skipping!
Property 'summary' already exists in node '9d8bb4'. Skipping!
Property 'summary' already exists in node 'f8a3ca'. Skipping!
Property 'summary' already exists in node '4fa6ae'. Skipping!
Property 'summary' already exists in node '840d82'. Skipping!


...

Property 'summary_embedding' already exists in node '7485bf'. Skipping!
Property 'summary_embedding' already exists in node 'e10c1d'. Skipping!
Property 'summary_embedding' already exists in node 'f8a3ca'. Skipping!
Property 'summary_embedding' already exists in node 'bc1cfb'. Skipping!
Property 'summary_embedding' already exists in node 'ea90f3'. Skipping!
Property 'summary_embedding' already exists in node '4fa6ae'. Skipping!
Property 'summary_embedding' already exists in node 'b4b10f'. Skipping!
Property 'summary_embedding' already exists in node 'ad6808'. Skipping!
Property 'summary_embedding' already exists in node '05acc4'. Skipping!
Property 'summary_embedding' already exists in node 'ae453a'. Skipping!
Property 'summary_embedding' already exists in node '840d82'. Skipping!
Property 'summary_embedding' already exists in node '9d8bb4'. Skipping!
Property 'summary_embedding' already exists in node 'f74c8e'. Skipping!
Property 'summary_embedding' already exists in node 'ea06fc'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the School Participation D...,"[Chapter 1 Academic Years, Academic Calendars,...",The School Participation Division oversees the...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(a) specify regarding th...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) specifies the minimum number o...,single_hop_specifc_query_synthesizer
2,What information does Volume 8 provide regardi...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 states that clinical work conducted o...,single_hop_specifc_query_synthesizer
3,FWS is it a payment period?,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,How do the regulations governing instructional...,[<1-hop>\n\nboth the credit or clock hours and...,The regulations specify that for clock-hour an...,multi_hop_abstract_query_synthesizer
5,"How do the academic calendar requirements, inc...","[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic calendar requirements stipulate t...,multi_hop_abstract_query_synthesizer
6,Considering the requirements for initial disbu...,[<1-hop>\n\nboth the credit or clock hours and...,The eligibility for Pell Grant and TEACH Grant...,multi_hop_abstract_query_synthesizer
7,How do regulations governing instructional tim...,[<1-hop>\n\nboth the credit or clock hours and...,The regulations specify that disbursement of T...,multi_hop_abstract_query_synthesizer
8,Volume 2 and Volume 8 what about academic year...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","Volume 2 talks about academic years, weeks of ...",multi_hop_specific_query_synthesizer
9,"In Volume 8, how does inclusion of clinical wo...",[<1-hop>\n\nInclusion of Clinical Work in a St...,"According to Volume 8, clinical work included ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [19]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [20]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [21]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [24]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [26]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [29]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (which include student Federal PLUS Loans and parent Direct PLUS Loans on behalf of dependent students)\n\nAdditionally, there are Subsidized and Unsubsidized Federal Stafford Loans, and Federal SLS Loans and Federal PLUS Loans that were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010, but no new loans have been made under that FFEL Program since then.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [30]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [31]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: Evaluates the correctness of the answer compared to the reference answer.
- `labeled_helpfulness_evaluator`: Evaluates whether the response is helpful to the user, taking into account the correct reference answer.
- `empathy_evaluator`: Evaluates whether the response is empathetic and makes the user feel heard.

## LangSmith Evaluation

In [32]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'new-money-19' at:
https://smith.langchain.com/o/2001a951-f0eb-4ab5-b232-67fa82db4d9e/datasets/a8872156-0388-4a26-b95d-627722d9cb05/compare?selectedSessions=b9d7a280-b9f7-4039-b98f-66394bf66b48




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B provide guida...,I don't know.,None,Appendix B offers detailed guidance on disburs...,0,0,0,0.991753,aebe3766-eccb-4986-8d24-36c9f2334e32,147f246a-c793-4563-b7c4-d7ca88062a96
1,How do Volume 2 and Volume 8 of the academic r...,I don't know.,None,Volume 2 discusses the requirements for establ...,0,0,0,0.695691,6577624b-d837-4b89-b617-1ef7d01aff83,56733c1a-c9d3-441a-b633-0d16c7a249ac
2,"In Volume 8, how does inclusion of clinical wo...","Based on the provided context from Volume 3, V...",None,"According to Volume 8, clinical work included ...",1,1,0,8.083198,df976315-549a-4103-8d5f-7559fb9a0dde,991ecb20-26a9-4327-a3e6-636ca37d8d2f
3,Volume 2 and Volume 8 what about academic year...,I don't know.,None,"Volume 2 talks about academic years, weeks of ...",0,0,0,0.811447,552a5c70-3361-4d5e-b0b1-cf04f18faef2,b7beff0f-bf4b-4c3a-909c-bf4b74f7e8d5
4,How do regulations governing instructional tim...,"Based on the provided context, regulations gov...",None,The regulations specify that disbursement of T...,1,1,0,7.368166,92bd4bde-7e19-46ff-b604-6dbe01e7848e,3bc49ff5-4b34-4871-953c-3f0c8b3994b0
5,Considering the requirements for initial disbu...,Based on the provided context:\n\n1. **Credit ...,None,The eligibility for Pell Grant and TEACH Grant...,1,0,0,12.386756,d4940a09-01c1-4df7-a2f2-3f1b2364735a,b567dcec-42ec-4691-9d03-5a44684d89ef
6,"How do the academic calendar requirements, inc...",The academic calendar requirements ensure comp...,None,The academic calendar requirements stipulate t...,1,1,0,4.611872,812e4658-f106-4428-a0d9-976f7488163b,03418e46-d793-4caf-8673-648caa195be0
7,How do the regulations governing instructional...,The regulations ensure compliance with Title I...,None,The regulations specify that for clock-hour an...,1,0,0,8.080492,2931a148-bb3e-412e-9d32-7a2f22298f7e,9dc77d88-6c27-40f2-98ae-c3807160fd30
8,FWS is it a payment period?,"Based on the provided context, FWS (Federal Wo...",None,The payment period is applicable to all Title ...,1,1,0,2.553106,1d1fc5bd-2bae-4d26-b540-6f7bb5617ba1,bae80499-4286-4420-913d-b3f30f090fef
9,What information does Volume 8 provide regardi...,"Volume 8, Chapter 3 provides additional guidan...",None,Volume 8 states that clinical work conducted o...,0,0,0,1.636623,ee0c91dc-3116-43ac-b915-9909054e5cb0,16bb43f4-be6d-4f6d-98b2-a5d182dfc366


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [33]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [34]:
rag_documents = docs

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer: Larger chunk sizes provide more context per chunk allowing the retriever to capture more complete inofrmation and relationships, which helps the LLM generate more comprehensive and accurate answers, but may also include more irrelevant information that could confuse the model.

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer: Upgrading from text-embedding-3-small to text-embedding-3-large provides higher-dimensional, more nuanced embeddings that can capture semantic relationships and context more accurately, leading to better document retrieval and more relevant context for the LLM to generate answers.

In [37]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [38]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's really important to understand the types of loans available as you navigate financial aid options. Based on the information provided in the context, here are the kinds of loans available:\n\n1. **Direct Subsidized Loans** – These loans are for students with financial need and the government pays the interest while the student is in school at least half-time. The loan amount is limited to the student's financial need, which is calculated as the Cost of Attendance (COA) minus the student's Self-Help Aid (SAI) and Other Financial Aid (OFA).\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need, and interest accrues while the student is in school. Students may qualify for a combination of Direct Subsidized and Direct Unsubsidized Loans.\n\n3. **Direct PLUS Loans** – These loans are for parents of dependent undergraduate students or for graduate/professional students themselves to help cover the COA. There 

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'slight-condition-97' at:
https://smith.langchain.com/o/2001a951-f0eb-4ab5-b232-67fa82db4d9e/datasets/a8872156-0388-4a26-b95d-627722d9cb05/compare?selectedSessions=0552e223-385e-4122-a819-cadd9ee67dae




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B provide guida...,Thank you for your thoughtful question. Based ...,None,Appendix B offers detailed guidance on disburs...,0,0,1,4.402997,aebe3766-eccb-4986-8d24-36c9f2334e32,1197fd37-18f7-4fbf-96d1-3acb5ef46f52
1,How do Volume 2 and Volume 8 of the academic r...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the requirements for establ...,0,0,1,5.748237,6577624b-d837-4b89-b617-1ef7d01aff83,856a51f2-5936-4f50-9802-6fc449f645cb
2,"In Volume 8, how does inclusion of clinical wo...",Thank you for your thoughtful question. Based ...,None,"According to Volume 8, clinical work included ...",1,1,1,8.797555,df976315-549a-4103-8d5f-7559fb9a0dde,eeb1319b-0012-44c3-98e4-e0aa14112659
3,Volume 2 and Volume 8 what about academic year...,Thank you for your thoughtful question about h...,None,"Volume 2 talks about academic years, weeks of ...",1,1,1,6.801706,552a5c70-3361-4d5e-b0b1-cf04f18faef2,04bef609-761b-4ff8-9bff-44249b2392b7
4,How do regulations governing instructional tim...,Thank you for your thoughtful question. Based ...,None,The regulations specify that disbursement of T...,1,1,1,7.064808,92bd4bde-7e19-46ff-b604-6dbe01e7848e,0f1477b0-6370-4b5f-af22-7b2fd981d809
5,Considering the requirements for initial disbu...,Thank you for your thoughtful question. It’s c...,None,The eligibility for Pell Grant and TEACH Grant...,1,0,1,10.113976,d4940a09-01c1-4df7-a2f2-3f1b2364735a,347ae139-b43d-4e19-b777-b43ce8bad62c
6,"How do the academic calendar requirements, inc...",Thank you for your thoughtful question. It’s c...,None,The academic calendar requirements stipulate t...,1,1,1,5.951633,812e4658-f106-4428-a0d9-976f7488163b,833fb8d3-ae34-4e15-a887-a9c0094f9dab
7,How do the regulations governing instructional...,Thank you for your thoughtful question. I unde...,None,The regulations specify that for clock-hour an...,1,1,1,8.903718,2931a148-bb3e-412e-9d32-7a2f22298f7e,cd6cc8ac-dabf-451c-8383-cf6510d36f73
8,FWS is it a payment period?,Thank you for your question. Based on the cont...,None,The payment period is applicable to all Title ...,1,1,1,2.597784,1d1fc5bd-2bae-4d26-b540-6f7bb5617ba1,bcc5045f-9511-4b49-8d9f-5bfec1de8bdd
9,What information does Volume 8 provide regardi...,Thank you for your thoughtful question. Based ...,None,Volume 8 states that clinical work conducted o...,1,1,1,4.031404,ee0c91dc-3116-43ac-b915-9909054e5cb0,8561e759-295f-4404-88ba-9d285239b9bb


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

First Chain:
![Alt Text](./image.png)
Second Chain:
![Alt Text](./image2.png)